In [40]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages mysql:mysql-connector-java:8.0.22 pyspark-shell"

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from bs4 import BeautifulSoup
import requests
import certifi
import json 
import pandas as pd
base_url = 'http\\://localhost\\:8080/'
covid_all=os.path.join(base_url,'/api/covid/usa/')
from bs4 import BeautifulSoup
import requests
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
from datetime import datetime

r = requests.get('http://covid:8080/api/covid/world/countries/USA', auth=('admin', 'admin'))
response = r.json()

covid_monthly = pd.DataFrame.from_dict(response)

covid_monthly = covid_monthly.sort_values(by=['year', 'month'])

r = requests.get('http://covid:8080/api/unemploymentrate/race', auth=('admin', 'admin'))
response = r.json()
otherPeopleRDD = sc.parallelize(jsonStrings)
otherPeople = spark.read.json(otherPeopleRDD)
park_df = pd.DataFrame.from_dict(response)
print(park_df.dtypes)
#park_df['time'] = pd.to_datetime(park_df['submissionDate'], format='yyyy-mm-dd HH:MM:SS.ss')#'%d%b%Y:%H:%M:%S.%f')
#park_df['time'] = pd.to_datetime(park_df['year'].astype(str)  + park_df['month'], format='%Y%B')

combine_lambda = lambda x: datetime.date(int(x.year), int(x.month), 1)


af_df = park_df[(park_df['variableName']=='african american') & (park_df['year']>2019)]\
.sort_values(by=['year', 'month'])

af_df['time'] = pd.to_datetime(af_df[['year', 'month']].assign(day=1))
date_sub = af_df['time'].tolist()
print(date_sub)


af_df = af_df['unemployedRate'].tolist()


white_df = park_df[(park_df.variableName=='white') & (park_df['year']>2019)]\
.sort_values(by=['year', 'month'])['unemployedRate']\
.tolist()
print(white_df)


hisp_df = park_df[(park_df.variableName=='hispanic') & (park_df['year']>2019)]\
.sort_values(by=['year', 'month'])['unemployedRate']\
.tolist()
print(hisp_df)


asian = park_df[(park_df.variableName=='asian') & (park_df['year']>2019)]\
.sort_values(by=['year', 'month'])['unemployedRate']\
.tolist()


# # def build_date(m,y):
# #     return datetime.now().replace(day=1, month=int(m),year=int(y))
# # date_udf =udf(lambda m, y:build_date(m,y) , DateType())      
# combine_lambda = lambda x: '{}-{}'.format(x.year, x.month)
# # covid_new = covid_monthly.withColumn('submission_date',date_udf(covid_monthly.month,covid_monthly.year))\
# # .select('submission_date','monthlyNewCases').toPandas()
covid_deaths = covid_monthly['monthlyNewDeaths'].tolist()[:len(date_sub)]


id                  int64
variableType       object
variableName       object
year                int64
month               int64
unemployedRate    float64
submissionDate     object
dtype: object
[Timestamp('2020-01-01 00:00:00'), Timestamp('2020-02-01 00:00:00'), Timestamp('2020-03-01 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('2020-05-01 00:00:00'), Timestamp('2020-06-01 00:00:00'), Timestamp('2020-07-01 00:00:00'), Timestamp('2020-08-01 00:00:00'), Timestamp('2020-09-01 00:00:00'), Timestamp('2020-10-01 00:00:00'), Timestamp('2020-11-01 00:00:00'), Timestamp('2020-12-01 00:00:00'), Timestamp('2021-01-01 00:00:00'), Timestamp('2021-02-01 00:00:00')]
[3.4, 4.1, 13.8, 12.1, 10.2, 9.4, 7.5, 6.8, 5.7, 5.7, 5.9, 6.2, 5.9, 5.5]
[4.8, 6.3, 18.5, 17.2, 14.5, 13.0, 10.5, 10.0, 8.5, 8.3, 9.2, 9.4, 9.0, 8.2]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


labels = date_sub
#men_means = [20, 34, 30, 35, 27]
# women_means = [25, 32, 34, 20, 25]

x = np.arange(len(labels))  # the label locations
width = 0.20  # the width of the bars
plt.rcParams['figure.figsize'] = [35, 15]
plt.rcParams.update({'font.size': 24})
fig ,ax = plt.subplots()

rects1 = ax.bar(x -0.05, af_df, 0.10, label='african american')
rects2 = ax.bar(x + 0.05, white_df, .10, label='white')
rects3 = ax.bar(x - 0.15, hisp_df, .10, label='hispanic')
rects4 = ax.bar(x + 0.15, asian, .10, label='asian')


# # Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Unemployment rate')
ax.set_title('unemployment rate by sector')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)
autolabel(rects3)
autolabel(rects4)



ax2 = ax.twinx()
ax2.set_ylabel('Covid cases - Monthly - in Millions')

ax2.plot(covid_deaths,linewidth=6)

fig.tight_layout()

plt.show()